In [3]:
# import libraries
import requests
import http.client
import json
import pandas as pd
import datetime
from pandas.io.json import json_normalize
import csv

In [98]:
def get_API_credentials():
    
    """Retreive user specific API credentials hidden in a local file"""
    """ Return Tuple ('site_id', 'API_key', 'username', 'password') """
    
    # open csv file with context manager
    with open('credentials.csv') as file:
        
        #read contents of file
        reader = csv.reader(file, delimiter=',')
        
        # get first line of file
        creds = next(reader)
        
        #return credentials as tuple
        return tuple(creds)

In [111]:
def get_staff_user_token():
    
    """Obtain the staff user token for API requests"""
    
    # define connection
    conn = http.client.HTTPSConnection(BASE)

    # specify desired payload
    payload = "{\r\n\t\"Username\": \"_"+USER+"\",\r\n\t\"Password\": \""+PASS+"\"\r\n}"

    # define relevant credentials
    headers = {
        'Content-Type': "application/json",
        'Api-Key': API_KEY,
        'SiteId': SITE_ID
        }
    
    # make request to api
    conn.request("POST", "/public/v6/usertoken/issue", payload, headers)
    global API_count
    API_count += 1
    print('API calls: ', API_count)

    # get respnse from API
    res = conn.getresponse()
    data = res.read()

    # decode response
    user_data = data.decode("utf-8")
    #print(user_data)
    
    # identify staff user toke
    user_data_list = user_data.split('\"')
    staff_user_token = user_data_list[7]
    
    
    return staff_user_token

In [127]:
# define API parameters and credentials

# read in SM MASSAGE credentials from secret file
SITE_ID, API_KEY, USER, PASS = get_API_credentials()

# Sandbox testing credentials
test_SITE_ID ='-99'
test_user = 'Siteowner'
test_pass = 'apitest1234'
#SITE_ID, USER, PASS = test_SITE_ID, test_user, test_pass

# URL base
BASE = 'api.mindbodyonline.com'

# request staff user token
STAFF_USER_TOKEN = get_staff_user_token()

# file to hold all collected data
MASTER_FILE = 'all_appts.csv'

#API_count = 0

API calls:  53


In [128]:
get_staff_user_token()

API calls:  54


'e9fcc28118684e029e53646852140e565aba1baf624e4e29907f1e1ad37f2266'

In [163]:
def get_appointments(start, end):
    
    """Get appointmenst from a specified start and end date and return as pandas DataFrame"""
    """API limits to 100 appointments per call, and will truncate results"""
    
    # define connection
    conn = http.client.HTTPSConnection(BASE)
    
    # define relevan credentials and get user token
    headers = {
        'Api-Key': API_KEY,
        'SiteID': SITE_ID,
        'authorization': STAFF_USER_TOKEN
    }

    #start = '2019-12-01'
    #end = '2019-12-30'

    # specify API endpoints
    endpoint = 'appointment/staffappointments' +'?StartDate='+start   +'&EndDate='+end

    # make request
    conn.request('GET', '/public/v6/'+endpoint, headers=headers)
    global API_count
    API_count += 1
    
    # catch JSON repsonse
    res = conn.getresponse()
    data = res.read()
    string = data.decode('utf-8')
    json_obj = json.loads(string)
    #print(json_obj)
    
    # identify keys
    keys = json_obj.keys()
    keys = list(keys)
    #print(keys)
    
    # select appointments
    appts = json_obj[keys[1]]
    
    # build dataFrame
    df = pd.DataFrame(appts)
    
    # drop Resources column
    #df = df.drop('Resources', axis=1)
    
    return df


In [150]:
start_date = '2019-07-01'
end_date = '2019-07-01'
test_df = get_appointments(start_date, end_date)

In [151]:
test_df.head(50)

,GenderPreference,Duration,ProviderId,Id,Status,StartDateTime,EndDateTime,Notes,StaffRequested,ProgramId,SessionTypeId,LocationId,StaffId,ClientId,FirstAppointment,ClientServiceId
0,None,60,0,61583,Completed,2019-07-01T11:00:00,2019-07-01T12:00:00,,False,21,1107,1,3,100006060,True,101188
1,None,30,0,71569,Completed,2019-07-01T12:00:00,2019-07-01T12:30:00,None,False,2,1006,1,3,100006069,True,101006
2,None,60,0,71572,Completed,2019-07-01T12:45:00,2019-07-01T13:45:00,None,False,5,1024,1,3,100006068,True,101030
3,None,45,0,71583,Completed,2019-07-01T14:30:00,2019-07-01T15:15:00,,True,2,1010,1,3,100006074,True,101010
4,None,120,0,71581,Completed,2019-07-01T15:30:00,2019-07-01T17:30:00,,True,2,1020,1,3,100006073,True,101026
5,Female,45,0,61490,Completed,2019-07-01T17:30:00,2019-07-01T18:15:00,,True,21,1097,1,3,100006021,True,101156
6,None,60,0,71580,Completed,2019-07-01T19:00:00,2019-07-01T20:00:00,,True,5,1024,1,3,100006072,True,101030
7,None,30,0,71590,Completed,2019-07-01T17:15:00,2019-07-01T17:45:00,,False,2,1006,1,100000001,100006077,True,101006
8,None,45,0,71592,Completed,2019-07-01T18:15:00,2019-07-01T19:00:00,,False,4,1074,1,100000001,100005713,False,101126
9,None,45,0,71595,Completed,2019-07-01T19:00:00,2019-07-01T19:45:00,,False,2,1010,1,100000001,100006079,True,101010


In [152]:
def get_previous_appointments(num_days):
    """ return DataFrame with appointment data for the last N days"""
    
    # generate list of desired dates
    base = datetime.datetime.today()
    date_list = [base - datetime.timedelta(days=x) for x in range(num_days)]
    
    # empty list to hold single day appt DataFrames
    all_appts = []
    
    # loop through desired days
    for i, day in enumerate(date_list):
        
        if i%7==0:
            print("collecting data for ", day.date())
            
        # define date parameters for API request
        start = str(day.date())
        end = start
        
        # request a single day of data
        single_day_appts = get_appointments(start, end)
        
        #if single_day_appts != None:
        
        # convert single day to DataFrame
        single_day_df = pd.DataFrame(single_day_appts)
        
        # append single day to all data
        all_appts.append(single_day_appts)

    # concat the list of daily DataFrames
    appts_df = pd.concat(all_appts)

    return appts_df
        
        
    

In [153]:
test_df = get_previous_appointments(10)


collecting data for  2020-01-10
collecting data for  2020-01-03


In [156]:
test_df.head(50)

,GenderPreference,Duration,ProviderId,Id,Status,StartDateTime,EndDateTime,Notes,StaffRequested,ProgramId,SessionTypeId,LocationId,StaffId,ClientId,FirstAppointment,ClientServiceId
0,None,120,0,116940,NoShow,2020-01-10T11:30:00,2020-01-10T13:30:00,None,False,8,2124,1,3,100005595,False,NaN
1,None,45,0,116958,Completed,2020-01-10T19:00:00,2020-01-10T19:45:00,,True,21,1097,1,3,100004735,False,101156.0
2,None,10,0,116978,Completed,2020-01-10T21:00:00,2020-01-10T21:10:00,,False,6,1109,1,3,100008397,True,101191.0
3,None,45,0,116917,Completed,2020-01-10T14:00:00,2020-01-10T14:45:00,,False,2,1010,1,100000004,100006303,False,101102.0
4,None,60,0,116957,Completed,2020-01-10T11:15:00,2020-01-10T12:15:00,,False,5,1024,1,100000022,100007681,False,101030.0
5,None,60,0,116959,Completed,2020-01-10T13:30:00,2020-01-10T14:30:00,,False,4,1007,1,100000022,100008390,True,101007.0
6,None,30,0,116964,Completed,2020-01-10T15:15:00,2020-01-10T15:45:00,,False,2,1006,1,100000022,100007329,False,101006.0
7,None,10,0,116977,Completed,2020-01-10T21:00:00,2020-01-10T21:10:00,,False,6,1109,1,100000022,100008397,True,101191.0
8,None,60,0,116955,Completed,2020-01-10T13:00:00,2020-01-10T14:00:00,Wants strong\nReferred by Nick White,False,2,5,1,100000037,100008389,True,100003.0
9,None,30,0,116963,Completed,2020-01-10T14:45:00,2020-01-10T15:15:00,,False,4,2138,1,100000037,100008392,True,102236.0


In [160]:
def generate_data_file():
    
    """Create as csv file where new data will be appended"""
    """ DO NOT RUN: WILL DESTROY EXISTING DATA IN FILE"""
    
    get_previous_appointments(1).to_csv(MASTER_FILE)

# DO NOT RUN: WILL DESTROY EXISTING DATA IN FILE
generate_data_file()

collecting data for  2020-01-10


In [169]:
def append_data_file(num_days):
    
    """Collect data from recent days and append to the existing data"""
    
    # read existing data
    existing = pd.read_csv(MASTER_FILE, header=0)
    existing = existing.drop(existing.columns[0], axis=1)
    
    # get recent data
    new = get_previous_appointments(num_days)
    
    # concat new and existing data & drop ducpicate rows
    updated = pd.concat([existing, new], sort=True)
    
    # drop duplicate rows
    # updated = updated.drop_duplicates()
    
    updated.to_csv(MASTER_FILE)
    
    return updated      

In [188]:
updated_df = append_data_file(20)
print(updated_df.shape)
updated_df.head()

collecting data for  2020-01-10
collecting data for  2020-01-03
collecting data for  2019-12-27
(2111, 17)


,ClientId,ClientServiceId,Duration,EndDateTime,FirstAppointment,GenderPreference,Id,LocationId,Notes,ProgramId,ProviderId,Resources,SessionTypeId,StaffId,StaffRequested,StartDateTime,Status
0,100005595,NaN,120,2020-01-10T13:30:00,False,None,116940,1,NaN,8,0,NaN,2124,3,False,2020-01-10T11:30:00,NoShow
1,100004735,101156.0,45,2020-01-10T19:45:00,False,None,116958,1,NaN,21,0,NaN,1097,3,True,2020-01-10T19:00:00,Completed
2,100008397,101191.0,10,2020-01-10T21:10:00,True,None,116978,1,NaN,6,0,NaN,1109,3,False,2020-01-10T21:00:00,Completed
3,100006303,101102.0,45,2020-01-10T14:45:00,False,None,116917,1,NaN,2,0,NaN,1010,100000004,False,2020-01-10T14:00:00,Completed
4,100007681,101030.0,60,2020-01-10T12:15:00,False,None,116957,1,NaN,5,0,NaN,1024,100000022,False,2020-01-10T11:15:00,Completed


In [194]:
drop_resource = updated_df.drop('Resources', axis=1)
print(drop_resource.shape)
clean = drop_resource.drop_duplicates(keep='first')
clean.to_csv('clean.csv')
clean.shape

(2111, 16)


(624, 16)

In [284]:
def get_staff():
    
    """ Request staff details and return as DataFrame """
    conn = http.client.HTTPSConnection(BASE)
    headers = {
        'Api-Key': API_KEY,
        'SiteID': SITE_ID,
        'authorization': STAFF_USER_TOKEN
    }

    endpoint = 'staff/staff'

    conn.request('GET', '/public/v6/'+endpoint, headers=headers)
    global API_count
    API_count += 1
    res = conn.getresponse()
    data = res.read()
    string = data.decode('utf-8')
    json_obj = json.loads(string)
    
    keys = json_obj.keys()
    keys = list(keys)
    print(keys)
    
    
    x = json_obj[keys[1]]
    
    df = pd.DataFrame(x)
    
    return df

In [285]:
def get_staff_ids():
    
    """ Return dict of staff ids and employee names"""
    
    # request staff data
    staff = get_staff()
    
    # create FullName column
    staff['FullName'] = staff.FirstName.astype(str) + staff.LastName.astype(str)
    
    # contruct dict from name & id pairs
    staff_ids = pd.Series(staff.FullName.values, index=staff.Id).to_dict()
    
    return staff_ids

In [287]:
def get_bookables(session_type_id):
    
    """ Return a Dict of features for an individual bookable session"""
    
    conn = http.client.HTTPSConnection(BASE)
    headers = {
        'Api-Key': API_KEY,
        'SiteID': SITE_ID,
        'authorization': STAFF_USER_TOKEN
    }

    endpoint = 'appointment/bookableitems?SessionTypeIds=' + str(session_type_id)

    conn.request('GET', '/public/v6/'+endpoint, headers=headers)
    global API_count
    API_count += 1
    res = conn.getresponse()
    data = res.read()
    string = data.decode('utf-8')
    json_obj = json.loads(string)
    
    keys = json_obj.keys()
    keys = list(keys)
    #print(keys)
    
    
    x = json_obj[keys[1]]
    #print(x)
    
    #df = pd.DataFrame(x)
    
    return x

In [288]:
def session_names(session_ids):
    
    """ Build a dictionary of session ids from a provided list of session_ids """
    
    # instantiate empty session name dict
    id_names = {}
    
    # loop through session ids
    for session in session_ids:
        
        # get session details
        bookables = get_bookables(session)
        features = bookables[0]['SessionType']
        
        # build dict with session_id and session_name
        id_names[features['Id']] = features['Name']
    
    return id_names

In [289]:
#THE BREAD AND BUTTER
#all_data = append_data_file(649)
#all_data.to_csv('duplicate_data.csv')

In [296]:
def main():
    
    """ The Main method that builds, wrangles, and cleans the DataFrame"""


    # make copy of all appointment data
    #appts_df = all_data.copy()
    appts_df = pd.read_csv(MASTER_FILE, header=0)

    # map Employee Names to StaffId
    appts_df['Staff'] = appts_df['StaffId'].map(get_staff_ids())

    # convert to DateTimes
    appts_df.StartDateTime = pd.to_datetime(appts_df.StartDateTime)
    appts_df.EndDateTime = pd.to_datetime(appts_df.EndDateTime)

    # Identify unique Sesseion Type Ids
    session_types = appts_df.SessionTypeId.unique()
    appts_df['SessionName'] = appts_df.SessionTypeId.map(session_names(session_types))
    
    
    appts_df.to_csv('clean_data.csv')
    
    appts.head(30)



In [295]:
clean_data = main_function()
clean_data.head(30)

['PaginationResponse', 'StaffMembers']


,Unnamed: 0,GenderPreference,Duration,ProviderId,Id,Status,StartDateTime,EndDateTime,Notes,StaffRequested,ProgramId,SessionTypeId,LocationId,StaffId,ClientId,FirstAppointment,ClientServiceId,Staff,SessionName
0,0,None,90,0,116945,Arrived,2020-01-09 15:30:00,2020-01-09 17:00:00,NaN,True,2,1011,1,3,100000811,False,101109.0,LinLi,Deep Tissue - 90 Minute
1,1,None,60,0,116946,NoShow,2020-01-09 18:00:00,2020-01-09 19:00:00,NaN,False,2,5,1,3,100000363,False,NaN,LinLi,Deep Tissue - 60 Minute
2,2,None,30,0,116942,Completed,2020-01-09 12:45:00,2020-01-09 13:15:00,NaN,False,2,1006,1,100000022,100007622,False,101006.0,ZhongcaiZheng,Deep Tissue - 30 Minute
3,3,None,180,0,116503,NoShow,2020-01-09 11:00:00,2020-01-09 14:00:00,NaN,False,8,1123,1,100000033,100000599,False,NaN,AndrewHalarewicz,Team Meeting - 60
4,4,None,60,0,116943,Arrived,2020-01-09 14:45:00,2020-01-09 15:45:00,NaN,False,21,1111,1,100000037,100006196,False,NaN,MichaelButler,Bio Electric Massage - 45
5,5,None,30,0,116909,Completed,2020-01-09 11:00:00,2020-01-09 11:30:00,NaN,False,2,1006,1,100000061,100008373,True,101006.0,XiaolingSitu,Deep Tissue - 30 Minute
6,6,None,90,0,116945,Arrived,2020-01-09 15:30:00,2020-01-09 17:00:00,NaN,True,2,1011,1,3,100000811,False,101109.0,LinLi,Deep Tissue - 90 Minute
7,7,None,60,0,116946,NoShow,2020-01-09 18:00:00,2020-01-09 19:00:00,NaN,False,2,5,1,3,100000363,False,NaN,LinLi,Deep Tissue - 60 Minute
8,8,None,30,0,116942,Completed,2020-01-09 12:45:00,2020-01-09 13:15:00,NaN,False,2,1006,1,100000022,100007622,False,101006.0,ZhongcaiZheng,Deep Tissue - 30 Minute
9,9,None,180,0,116503,NoShow,2020-01-09 11:00:00,2020-01-09 14:00:00,NaN,False,8,1123,1,100000033,100000599,False,NaN,AndrewHalarewicz,Team Meeting - 60
